In [1]:
%pylab inline
%matplotlib inline

import os
import time
import collections
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")
sys.path.append("/Users/Bya/git/predictEPL/WebScrapping/")

import paths
import espn_urls
import my_plot
import scrap_espn_gamecast
import emolex
import useful_methods

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

dfGameInfos.tail()

,GW,away_team,date,home_team,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time
195,20,Swansea,Sat. 2 Jan.,United,1,2,0,0,15:00
196,20,Bournemouth,Sat. 2 Jan.,Leicester,0,0,0,0,15:00
197,20,City,Sat. 2 Jan.,Watford,2,1,0,0,17:30
198,20,Chelsea,Sun. 3 Jan.,Crystal,3,0,1,0,13:30
199,20,Tottenham,Sun. 3 Jan.,Everton,1,1,1,1,16:00


In [3]:
# HT: Equal games
# dfGameInfosEqual = dfGameInfos[(dfGameInfos['score_ht_home'] == dfGameInfos['score_ht_away']) &
#                                (dfGameInfos['GW'] >= 4)].copy()

weeks = list(dfGameInfos['GW'])
home_teams = list(dfGameInfos['home_team'])
away_teams = list(dfGameInfos['away_team'])
score_ht_homes = list(dfGameInfos['score_ht_home'])
score_ht_aways = list(dfGameInfos['score_ht_away'])
score_ft_homes = list(dfGameInfos['score_ft_home'])
score_ft_aways = list(dfGameInfos['score_ft_away'])

In [4]:
# Read Emotion-Lexicon-Soccer as Dictionary
dic_emolex_soccer, dic_emolex_stemmed_soccer = emolex.EmolexSoccerDic()

All Words: 14162


In [5]:
def EmolexPN(df, start=1, end=45):
    dfEmolex = df.copy()
    
    # set interval
    ith_minute_start = start
    ith_minute_end = end

    # convert to integer
    dfEmolex['ith_minute'] = [int(number) for number in dfEmolex['ith_minute']]
    dfEmolex['positive'] = [int(number) for number in dfEmolex['positive']]
    dfEmolex['negative'] = [int(number) for number in dfEmolex['negative']]

    # Minute Interval's DF
    dfEmolex = dfEmolex[
        (dfEmolex['ith_minute'] >= ith_minute_start) &
        (dfEmolex['ith_minute'] <= ith_minute_end)
    ].copy()

    # Counting pos, neg
    positive = sum(list(dfEmolex['positive']))
    negative = sum(list(dfEmolex['negative']))
    
    # Return Positve Percentage
    return (positive / (positive + negative))

In [6]:
res_ht_rt = []

start_time = time.time()

for index in range(len(weeks)):
    try:
        print(index)

        # Read Single Game as DF
        filename = home_teams[index] + '_vs_' + away_teams[index] + '.csv'
        os.chdir(paths.READ_PATH_EXTRACTED_CSV + 'GW' + str(weeks[index]) + '/SingleGames/')
        df = useful_methods.csv_dic_df(filename)

        # Filter DF: remove tweets of stream, bots ...
        dfFilter = useful_methods.FilterDF(df)

        # Count Emolex cateries on Tweets by minutes
        dfFilterEmolexSoccerHome = emolex.CreateEmolexDF(dfFilter[(dfFilter['side'] == 'home') & (dfFilter['status'] != 'retweet')], dic_emolex_soccer, dic_emolex_stemmed_soccer)
        dfFilterEmolexSoccerAway = emolex.CreateEmolexDF(dfFilter[(dfFilter['side'] == 'away') & (dfFilter['status'] != 'retweet')], dic_emolex_soccer, dic_emolex_stemmed_soccer)

        # Calculate PN %
        emolex_pn_home = EmolexPN(dfFilterEmolexSoccerHome)
        emolex_pn_away = EmolexPN(dfFilterEmolexSoccerAway)

        # Append Resulst
        res_ht_rt.append([index, emolex_pn_home, emolex_pn_away])
    except:
        continue

print("[Done]: %.2f" % (time.time() - start_time))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
[Done]: 1402.58


In [7]:
ok = 0
ng = 0
eq = 0

for res in res_ht_rt:
    index, pn_home, pn_away = res
    
    if (pn_home > pn_away) and (score_ft_homes[index] > score_ft_aways[index]):
        ok += 1
        print("\n------------------------------")
        print("[GAME]: GW%s %s vs %s" %(weeks[index], home_teams[index], away_teams[index]))
        print("[SCORE]: HT(%s-%s) => FT(%s-%s)" % (score_ht_homes[index], score_ht_aways[index],
                                          score_ft_homes[index], score_ft_aways[index]))
        print("\t[Emolex PN Score]: %s vs %s" % (pn_home, pn_away))

    elif (pn_home < pn_away) and (score_ft_homes[index] < score_ft_aways[index]):
        ok += 1
        print("\n------------------------------")
        print("[GAME]: GW%s %s vs %s" %(weeks[index], home_teams[index], away_teams[index]))
        print("[SCORE]: HT(%s-%s) => FT(%s-%s)" % (score_ht_homes[index], score_ht_aways[index],
                                          score_ft_homes[index], score_ft_aways[index]))
        print("[Emolex PN Score]: %s vs %s" % (pn_home, pn_away))
    
    elif score_ft_homes[index] == score_ft_aways[index]:
        eq += 1
    else:
        ng += 1

print("\n\n\nOK: %s, NG: %s, EQ: %s" % (ok, ng, eq))


------------------------------
[GAME]: GW4 Newcastle vs Arsenal
[SCORE]: HT(0-0) => FT(0-1)
[Emolex PN Score]: 0.42150680385 vs 0.489927132447

------------------------------
[GAME]: GW4 Stoke vs WestBromwich
[SCORE]: HT(0-1) => FT(0-1)
[Emolex PN Score]: 0.307692307692 vs 0.414201183432

------------------------------
[GAME]: GW4 Liverpool vs WestHam
[SCORE]: HT(0-2) => FT(0-3)
[Emolex PN Score]: 0.524477918521 vs 0.73576309795

------------------------------
[GAME]: GW4 Chelsea vs Crystal
[SCORE]: HT(0-0) => FT(1-2)
[Emolex PN Score]: 0.58137510879 vs 0.746478873239

------------------------------
[GAME]: GW4 Southampton vs Norwich
[SCORE]: HT(1-0) => FT(3-0)
	[Emolex PN Score]: 0.582872928177 vs 0.47905027933

------------------------------
[GAME]: GW4 Swansea vs United
[SCORE]: HT(0-0) => FT(2-1)
	[Emolex PN Score]: 0.7075 vs 0.673735976641

------------------------------
[GAME]: GW5 Everton vs Chelsea
[SCORE]: HT(2-1) => FT(3-1)
	[Emolex PN Score]: 0.615615615616 vs 0.48474894415

In [8]:
# Save Resulst as CSV
dfResult = dfGameInfos.copy()

def PnScore(index, side):
    for res in res_ht_rt:
        if index == res[0]:
            return res[side]
    return -1

dfResult['pn_away'] = [PnScore(index, 2) for index in range(len(weeks))]
dfResult['pn_home'] = [PnScore(index, 1) for index in range(len(weeks))]


# Save as CSV
useful_methods.DFtoCSV(dfResult, paths.READ_PATH_GAME_INFO, 'Predict-Emolex-45min-NonRT', index=False)
print("[Saved in]: %s" % (paths.READ_PATH_GAME_INFO + 'Predict-Emolex-45min-NonRT.csv'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/Predict-Emolex-45min-NonRT.csv


In [ ]:
def PnAnalysis(dfResult):
    # Predicted: OK
    predict_good = len(dfResult[
        (dfResult['pn_home'] != -1) &

        # Predict: GOOD
        (
            # PN: Home > Away & FT: Home > Away
            ((dfResult['pn_home'] > dfResult['pn_away']) & (dfResult['score_ft_home'] > dfResult['score_ft_away'])) |
            # PN: Home < Away & FT: Home < Away
            ((dfResult['pn_home'] < dfResult['pn_away']) & (dfResult['score_ft_home'] < dfResult['score_ft_away']))
        )
    ])
    print("[PREDICTED GOOD]: %s" %(predict_good))

    # Predicted: NG
    predict_ng = len(dfResult[
        (dfResult['pn_home'] != -1) &

        # Predict:  NOT GOOD
        (
            # PN: Home > Away & FT: Home < Away
            ((dfResult['pn_home'] > dfResult['pn_away']) & (dfResult['score_ft_home'] < dfResult['score_ft_away'])) |
            # PN: Home < Away & FT: Home > Away
            ((dfResult['pn_home'] < dfResult['pn_away']) & (dfResult['score_ft_home'] > dfResult['score_ft_away']))
        )
    ])
    print("[PREDICTED NG]: %s" %(predict_ng))

    # FT : Equal
    ft_equal = len(dfResult[
        (dfResult['pn_home'] != -1) &

        # FT: Home == Away
        (dfResult['score_ft_home'] == dfResult['score_ft_away'])
    ])
    print("[Full-Time Equal]: %s" %(ft_equal))

    return predict_good, predict_ng, ft_equal


PnAnalysis(dfResult)
print("\n")

PnAnalysis(dfResult[(dfResult['score_ht_home'] == dfResult['score_ht_away'])])
print("\n")

PnAnalysis(dfResult[(dfResult['score_ht_home'] > dfResult['score_ht_away'])])
print("\n")

PnAnalysis(dfResult[(dfResult['score_ht_home'] < dfResult['score_ht_away'])])
print("\n")